In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from time import perf_counter
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from IPython.display import Markdown, display
def printmd(string):
    # Print with Markdowns    
    display(Markdown(string))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df=pd.read_csv('../input/phishing-website-dataset/dataset.csv')

In [3]:
df.head(3)

In [4]:
df.shape

In [5]:
import seaborn as sns
sns.countplot(df['Result'])

In [6]:
col=df.columns
for i in col:
     if  i!='index':
        print(i,df[i].unique())


In [7]:
df['Result']=df['Result'].replace(-1,0)

In [8]:
print(df.corr()['Result'].sort_values())    

In [9]:
plt.figure(figsize=(8, 8))
sns.heatmap(df.corr(), linewidths=.5)

In [10]:
from sklearn.model_selection import train_test_split,cross_val_score

X= df.drop(columns='Result')
Y=df['Result']

train_X,test_X,train_Y,test_Y=train_test_split(X,Y,test_size=0.3,random_state=2)
Accuracy=[]
Duration=[]

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
start = perf_counter()
logreg=LogisticRegression(solver='liblinear')
logreg=logreg.fit(train_X,train_Y)
prediction=logreg.predict(test_X)
accuracy_score(prediction,test_Y)
Accuracy.append(accuracy_score(prediction,test_Y))
print(Accuracy[0])

duration = perf_counter() - start
duration = round(duration,2)
Duration.append(duration)
print(Duration[0])

In [12]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
start = perf_counter()
clf = BernoulliNB()
clf.fit(train_X, train_Y)
naive_predict=clf.predict(test_X)
accuracy_score(naive_predict,test_Y)
Accuracy.append(accuracy_score(naive_predict,test_Y))
print(Accuracy[1])

duration = perf_counter() - start
duration = round(duration,2)
Duration.append(duration)
print(Duration[1])

In [13]:
from sklearn.ensemble import RandomForestClassifier
start = perf_counter()
forest_clf = RandomForestClassifier(max_depth=10, random_state=0)
forest_clf.fit(train_X,train_Y)
ran_pred=forest_clf.predict(test_X)
accuracy_score(ran_pred,test_Y)
Accuracy.append(accuracy_score(ran_pred,test_Y))
print(Accuracy[2])

duration = perf_counter() - start
duration = round(duration,2)
Duration.append(duration)
print(Duration[2])

In [14]:
from sklearn.tree import DecisionTreeClassifier
start = perf_counter()
tree = DecisionTreeClassifier(criterion='entropy') # function to measure the quality of split 
tree.fit(train_X,train_Y)
score = cross_val_score(tree, X, Y, cv= 10) 
print(score)
print(score.mean())
y_pred = tree.predict(test_X)
Accuracy.append(accuracy_score(y_pred,test_Y))
print(Accuracy[3])

duration = perf_counter() - start
duration = round(duration,2)
Duration.append(duration)
print(Duration[3])

In [15]:
model=["LogisticRegression", "BernoulliNB","RandomForestClassifier", "DecisionTreeClassifier"]
models_acc = []
for i in range(0,4):
    models_acc.append([model[i],Accuracy[i],Duration[i]])
df_acc = pd.DataFrame(models_acc)
df_acc.columns = ['Model', 'Accuracy', 'Training time (sec)']
df_acc.sort_values(by = 'Accuracy', ascending = False, inplace=True)
df_acc.reset_index(drop = True, inplace=True)
df_acc

In [16]:
plt.figure(figsize = (15,5))
sns.barplot(x = 'Model', y = 'Accuracy', data = df_acc)
plt.title('Accuracy on the test set\n(the Y-Axis is between 0.8 and 1.0)', fontsize = 15)
plt.ylim(0.85,1)
plt.show()

In [17]:
plt.figure(figsize = (15,5))
sns.barplot(x = 'Model', y = 'Training time (sec)', data = df_acc)
plt.title('Training time for each model in sec', fontsize = 15)
plt.ylim(0,0.7)
plt.show()

In [18]:
tree = DecisionTreeClassifier(criterion='entropy') # function to measure the quality of split 
tree.fit(train_X,train_Y)
y_pred = tree.predict(test_X)
def display_result(df):
    for i in range (0,20,4):
        msg = df.drop(columns='Result')
        label = df["Result"].iloc[i]
        pred_label = tree.predict(msg)
        printmd(f"**Real: {label}, Predicted: {pred_label[i]}**")
        printmd("_______________________________________________________________")
        print()
        print()
display_result(df)   